In [1]:
import os, sys
from pathlib import Path
import cortex
import nibabel as nib
import numpy as np
import matplotlib.colors as colors
import matplotlib.pyplot as pl
import time
import platform

In [2]:
if platform.node()=='theCryingMachine.local':
    deriv_dir = Path('/Users/ningkai/Library/CloudStorage/OneDrive-Personal/Workbench/suppr_pRF/data/spinoza/sub-002/derivatives/')
elif platform.node()=='Smoldering-Corpse-Bar.local':
    deriv_dir = Path('/Users/dionysus/Library/CloudStorage/OneDrive-Personal/Workbench/suppr_pRF/data/spinoza/sub-002/derivatives/')
elif platform.node()=='node230':
    raise ValueError('hmmm I dont think it works on a server. Your are working on {}'.format(platform.node()))
else:
    raise ValueError('No idea which computer this is. The computer name is {}'.format(platform.node()))

print('Running on {}'.format(platform.node()))
print('Current deriv folder is {}'.format(deriv_dir))
data_dir = Path(deriv_dir, 'GLMsingle')


Running on Smoldering-Corpse-Bar.local
Current deriv folder is /Users/dionysus/Library/CloudStorage/OneDrive-Personal/Workbench/suppr_pRF/data/spinoza/sub-002/derivatives


In [3]:
print('cortex.database.default_filestore: {}'.format(cortex.database.default_filestore))
print('cortex.options.usercfg: {}'.format(cortex.options.usercfg))

cortex.database.default_filestore: /Users/dionysus/Documents/pycortex/db
cortex.options.usercfg: /Users/dionysus/Library/Application Support/pycortex/options.cfg


In [4]:
# import data into pycortex
subj_nr = 2
ses_nr = 1
sub = "sub-%s" % str(subj_nr).zfill(3)
ses = "ses-%s" % str(ses_nr).zfill(2)
fs_dir = Path(deriv_dir, 'freesurfer')
fs_subj_dir = Path(fs_dir, sub)
# pial_path = Path(fs_subj_dir, 'surf', 'lh.pial')
ln_pial_path = Path(fs_subj_dir, 'surf', 'lh.pial')
# os.system("ln -s {} {}".format(pial_path, ln_pial_path))


xfmname = "fmriprep_fsT1"
dd = Path(data_dir, sub, ses)
epi = Path(dd, "reg", "epi.nii.gz")
epi_2_T1 = Path(dd, "reg", "bbr.mat")
# T1 = Path(deriv_dir, 'fmriprep', sub, ses, 'anat', 'sub-002_ses-01_acq-MP2RAGE_desc-preproc_T1w.nii.gz')
T1 = Path(fs_dir, sub, "mri", "T1.nii.gz")
os.system("mri_convert %s %s" % (Path(str(T1).replace('.nii.gz', '.mgz')), T1))

bg_image_fn = Path(deriv_dir, 'func', 'sub-002_ses-01_task-ping_run-6_space-T1w_boldref.nii.gz')
bm_image_fn = Path(deriv_dir, 'func', 'sub-002_ses-01_task-ping_run-6_space-T1w_desc-brain_mask.nii.gz')
bref = nib.load(bg_image_fn)
bmask = nib.load(bm_image_fn).get_fdata().astype(bool)
print('bref.shape: {}'.format(bref.shape))
print('bmask.shape: {}'.format(bmask.shape))

bref.shape: (91, 106, 86)
bmask.shape: (91, 106, 86)


In [5]:
np.identity(4)

array([[1., 0., 0., 0.],
       [0., 1., 0., 0.],
       [0., 0., 1., 0.],
       [0., 0., 0., 1.]])

In [6]:
bref.affine

array([[   1.68799996,    0.        ,    0.        ,  -76.11100006],
       [   0.        ,    1.68799996,    0.        , -101.10510254],
       [   0.        ,    0.        ,    1.70000005,  -58.36110306],
       [   0.        ,    0.        ,    0.        ,    1.        ]])

In [19]:
a = np.identity(4)
transform2 = cortex.xfm.Transform(np.identity(4), bref)
print(transform2.xfm)
print(bref.shape)
print(transform2.shape)

[[1. 0. 0. 0.]
 [0. 1. 0. 0.]
 [0. 0. 1. 0.]
 [0. 0. 0. 1.]]
(91, 106, 86)
(86, 106, 91)


In [10]:
cortex.freesurfer.import_subj(fs_subject=sub,
                                cx_subject=sub,
                                freesurfer_subject_dir=fs_dir)
# cortex.freesurfer.import_flat(fs_subject='sub-'+str(subj_nr).zfill(3),
#                                 patch='full',
#                                 freesurfer_subject_dir=fs_dir)
# add_session_2_cortex(epi_2_T1, epi, T1, sub, xfmname, transpose=False)

transform = cortex.xfm.Transform(np.identity(4), bref)
transform.save(sub, xfmname, 'magnet')

b'created by nwa201 on Tue Dec 12 21:21:24 2023\n'
b'created by nwa201 on Wed Dec 13 01:55:48 2023\n'
b'created by nwa201 on Tue Dec 12 23:48:23 2023\n'
b'created by nwa201 on Wed Dec 13 02:00:33 2023\n'
b'created by nwa201 on Tue Dec 12 21:21:24 2023\n'
b'created by nwa201 on Tue Dec 12 23:48:23 2023\n'
b'created by nwa201 on Wed Dec 13 01:55:48 2023\n'
b'created by nwa201 on Wed Dec 13 02:00:33 2023\n'
b'created by nwa201 on Tue Dec 12 21:23:11 2023\n'
b'created by nwa201 on Tue Dec 12 23:49:12 2023\n'


In [11]:
for xfm_mask in ["cortical", "thick", "thin", "nearest"]:
    print('dealing with mask: {}'.format(xfm_mask))
    mask = cortex.get_cortical_mask(subject=sub, xfmname=xfmname, type=xfm_mask)
    mask_img = nib.Nifti1Image(dataobj=mask.transpose((2,1,0)), affine=bref.affine, header=bref.header)
    print('mask_img.shape: {}'.format(mask_img.shape))
    mask_file = "{}/{}/transforms/{}/mask_{}.nii.gz".format(cortex.database.default_filestore, sub, xfmname, xfm_mask)
    mask_img.to_filename(mask_file)

dealing with mask: cortical
0.000%
0.044%
0.088%
0.132%
0.176%
0.220%
0.263%
0.307%
0.351%
0.395%
0.439%
0.483%
0.527%
0.571%
0.615%
0.659%
0.703%
0.746%
0.790%
0.834%
0.878%
0.922%
0.966%
1.010%
1.054%
1.098%
1.142%
1.186%
1.230%
1.273%
1.317%
1.361%
1.405%
1.449%
1.493%
1.537%
1.581%
1.625%
1.669%
1.713%
1.756%
1.800%
1.844%
1.888%
1.932%
1.976%
2.020%
2.064%
2.108%
2.152%
2.196%
2.239%
2.283%
2.327%
2.371%
2.415%
2.459%
2.503%
2.547%
2.591%
2.635%
2.679%
2.722%
2.766%
2.810%
2.854%
2.898%
2.942%
2.986%
3.030%
3.074%
3.118%
3.162%
3.206%
3.249%
3.293%
3.337%
3.381%
3.425%
3.469%
3.513%
3.557%
3.601%
3.645%
3.689%
3.732%
3.776%
3.820%
3.864%
3.908%
3.952%
3.996%
4.040%
4.084%
4.128%
4.172%
4.215%
4.259%
4.303%
4.347%
4.391%
4.435%
4.479%
4.523%
4.567%
4.611%
4.655%
4.699%
4.742%
4.786%
4.830%
4.874%
4.918%
4.962%
5.006%
5.050%
5.094%
5.138%
5.182%
5.225%
5.269%
5.313%
5.357%
5.401%
5.445%
5.489%
5.533%
5.577%
5.621%
5.665%
5.708%
5.752%
5.796%
5.840%
5.884%
5.928%
5.972%
6.016%
6.060%

In [17]:
bref_cortex = cortex.Volume(bref.get_fdata().T, sub, xfmname, cmap='hot')
cortex.webshow(bref_cortex)

Started server on port 56263
Stopping server


<JS: window.viewer>

In [12]:
TYPED_R2_path = Path(data_dir, 'TYPED_R2.nii.gz')
TYPED_R2 = nib.load(TYPED_R2_path).get_fdata().T
TYPED_R2_cortex = cortex.Volume(TYPED_R2, sub, xfmname, cmap='hot') # cmap can be 'BuBkRd' for diversity colormap

TYPED_FRACvalue_path = Path(data_dir, 'TYPED_FRACvalue.nii.gz')
TYPED_FRACvalue = nib.load(TYPED_FRACvalue_path).get_fdata().T
TYPED_FRACvalue_cortex = cortex.Volume(TYPED_FRACvalue, sub, xfmname, cmap='hot')

TYPEC_retinamap_path = Path(data_dir, 'TYPEC_retinamap.nii.gz')
TYPEC_retinamap = nib.load(TYPEC_retinamap_path).get_fdata().T
TYPEC_retinamap_cortex = cortex.Volume(TYPEC_retinamap, sub, xfmname, cmap='hsv')

In [13]:
print('TYPED_R2 range: {} to {}'.format(str(TYPED_R2.min().round(3)), str(TYPED_R2.max().round(3))))
# cortex.webshow(TYPED_R2_cortex)

TYPED_R2 range: 0.0 to 26.047


In [14]:
print('TYPED_FRACvalue range: {} to {}'.format(str(TYPED_FRACvalue.min().round(3)), str(TYPED_FRACvalue.max().round(3))))
cortex.webshow(TYPED_FRACvalue_cortex)

TYPED_FRACvalue range: 0.0 to 0.65
Generating new ctm file...
wm
wm
inflated
inflated
Started server on port 27926
Stopping server


<JS: window.viewer>

In [ ]:
print('TYPEC_retinamap range: {} to {}'.format(str(TYPEC_retinamap.min().round(3)), str(TYPEC_retinamap.max().round(3))))
cortex.webshow(TYPEC_retinamap_cortex)

In [ ]:
cortex.webgl.make_static(outpath=deriv_dir, data=TYPED_FRACvalue_cortex)